In [37]:
from data import uci_har, uci_preprocessing
from simulation.make_data import get_dataloader

from models import HARClassifier, VAE_1D

from training import har_train

import torch
from torch import nn, optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

from utils import AverageMeter, concatenate

from tqdm.auto import tqdm
from torch.optim.lr_scheduler import MultiStepLR
import os

from attacker import LinfPGDAttack

import copy

In [2]:
uci_preprocessing('/workspace/Dataset/TSData/uci_data/UCI_HAR_Dataset/', '/workspace/Dataset/TSData/uci_data/np')

[4 4 4 ... 1 1 1] Counter({5: 1407, 4: 1374, 3: 1286, 0: 1226, 1: 1073, 2: 986})
[4 4 4 ... 1 1 1] Counter({5: 537, 4: 532, 0: 496, 3: 491, 1: 471, 2: 420})


In [49]:
train_X = np.load('/workspace/Dataset/TSData/uci_data/np/np_train_x.npy')
test_X = np.load('/workspace/Dataset/TSData/uci_data/np/np_test_x.npy')
train_y = np.load('/workspace/Dataset/TSData/uci_data/np/np_train_y.npy')
test_y = np.load('/workspace/Dataset/TSData/uci_data/np/np_test_y.npy')

In [51]:
for i in range(6):
    print((train_y == i).sum())

1226
1073
986
1286
1374
1407


In [46]:
train_X.shape

(7352, 9, 128)

In [47]:
test_X.shape

(2947, 9, 128)

In [48]:
7352 + 2947

10299

In [ ]:
    np.save(os.path.join(save_path, "np_train_x.npy"), train_x)
    np.save(os.path.join(save_path, "np_train_y.npy"), train_y)
    np.save(os.path.join(save_path, "np_test_x.npy"), test_x)
    np.save(os.path.join(save_path, "np_test_y.npy"), test_y)

In [4]:
for sparsity in [0.7, 0.8, 0.9, 0.91, 0.93, 0.95, 0.97, 0.99, 0.993, 0.996, 0.999, 0.1, 0.3, 0.5, 0.6]:
    har_train(cuda_num=4, EPOCH=20, save_interval=5, resume=True, comp_ratio=sparsity, model_dir='./simulation/HAR_UCI/')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1	Train loss: 0.8330	Validation accuracy: 0.8440
Epoch 2	Train loss: 0.2637	Validation accuracy: 0.8757
Epoch 3	Train loss: 0.1488	Validation accuracy: 0.8850
Epoch 4	Train loss: 0.1237	Validation accuracy: 0.8787
Epoch 5	Train loss: 0.1286	Validation accuracy: 0.8409
Epoch 6	Train loss: 0.1255	Validation accuracy: 0.8960
Epoch 7	Train loss: 0.1020	Validation accuracy: 0.8907
Epoch 8	Train loss: 0.1057	Validation accuracy: 0.8920
Epoch 9	Train loss: 0.1021	Validation accuracy: 0.8960
Epoch 10	Train loss: 0.0979	Validation accuracy: 0.8917
Epoch 11	Train loss: 0.1005	Validation accuracy: 0.8950
Epoch 12	Train loss: 0.0920	Validation accuracy: 0.8946
Epoch 13	Train loss: 0.0965	Validation accuracy: 0.8713
Epoch 14	Train loss: 0.1026	Validation accuracy: 0.8820
Epoch 15	Train loss: 0.0904	Validation accuracy: 0.8903
Epoch 16	Train loss: 0.0915	Validation accuracy: 0.8863
Epoch 17	Train loss: 0.0838	Validation accuracy: 0.8950
Epoch 18	Train loss: 0.0865	Validation accuracy: 0.8647
E

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1	Train loss: 0.9015	Validation accuracy: 0.7530
Epoch 2	Train loss: 0.4505	Validation accuracy: 0.8590
Epoch 3	Train loss: 0.2847	Validation accuracy: 0.8289
Epoch 4	Train loss: 0.2070	Validation accuracy: 0.8712
Epoch 5	Train loss: 0.1564	Validation accuracy: 0.8773
Epoch 6	Train loss: 0.1445	Validation accuracy: 0.8993
Epoch 7	Train loss: 0.1333	Validation accuracy: 0.8957
Epoch 8	Train loss: 0.1265	Validation accuracy: 0.9053
Epoch 9	Train loss: 0.1206	Validation accuracy: 0.8913
Epoch 10	Train loss: 0.1155	Validation accuracy: 0.9007
Epoch 11	Train loss: 0.1153	Validation accuracy: 0.9060
Epoch 12	Train loss: 0.1181	Validation accuracy: 0.9030
Epoch 13	Train loss: 0.1193	Validation accuracy: 0.9043
Epoch 14	Train loss: 0.1042	Validation accuracy: 0.9083
Epoch 15	Train loss: 0.1058	Validation accuracy: 0.9000
Epoch 16	Train loss: 0.1101	Validation accuracy: 0.9057
Epoch 17	Train loss: 0.1092	Validation accuracy: 0.9023
Epoch 18	Train loss: 0.1060	Validation accuracy: 0.8993
E

  0%|          | 0/20 [00:00<?, ?it/s]

...

Epoch 1	Train loss: 0.1223	Validation accuracy: 0.8467
Epoch 2	Train loss: 0.0792	Validation accuracy: 0.9160
Epoch 3	Train loss: 0.0718	Validation accuracy: 0.9190
Epoch 4	Train loss: 0.0640	Validation accuracy: 0.9163
Epoch 5	Train loss: 0.0644	Validation accuracy: 0.8997
Epoch 6	Train loss: 0.0727	Validation accuracy: 0.9233
Epoch 7	Train loss: 0.0620	Validation accuracy: 0.9187
Epoch 8	Train loss: 0.0577	Validation accuracy: 0.9040
Epoch 9	Train loss: 0.0639	Validation accuracy: 0.9097
Epoch 10	Train loss: 0.0616	Validation accuracy: 0.9130
Epoch 11	Train loss: 0.0601	Validation accuracy: 0.9120
Epoch 12	Train loss: 0.0612	Validation accuracy: 0.9197
Epoch 13	Train loss: 0.0593	Validation accuracy: 0.9100
Epoch 14	Train loss: 0.0570	Validation accuracy: 0.9203
Epoch 15	Train loss: 0.0509	Validation accuracy: 0.9080
Epoch 16	Train loss: 0.0692	Validation accuracy: 0.9120
Epoch 17	Train loss: 0.0716	Validation accuracy: 0.9187
Epoch 18	Train loss: 0.0620	Validation accuracy: 0.9170
E

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1	Train loss: 0.2522	Validation accuracy: 0.8926
Epoch 2	Train loss: 0.0997	Validation accuracy: 0.8957
Epoch 3	Train loss: 0.0920	Validation accuracy: 0.9063
Epoch 4	Train loss: 0.0823	Validation accuracy: 0.9143
Epoch 5	Train loss: 0.0838	Validation accuracy: 0.9083
Epoch 6	Train loss: 0.0771	Validation accuracy: 0.8933
Epoch 7	Train loss: 0.0933	Validation accuracy: 0.9043
Epoch 8	Train loss: 0.0751	Validation accuracy: 0.9070
Epoch 9	Train loss: 0.0857	Validation accuracy: 0.9057
Epoch 10	Train loss: 0.0721	Validation accuracy: 0.9043
Epoch 11	Train loss: 0.0712	Validation accuracy: 0.9180
Epoch 12	Train loss: 0.1046	Validation accuracy: 0.8610
Epoch 13	Train loss: 0.0838	Validation accuracy: 0.9030
Epoch 14	Train loss: 0.0724	Validation accuracy: 0.9043
Epoch 15	Train loss: 0.0626	Validation accuracy: 0.9080
Epoch 16	Train loss: 0.0768	Validation accuracy: 0.9133
Epoch 17	Train loss: 0.0645	Validation accuracy: 0.9070
Epoch 18	Train loss: 0.0711	Validation accuracy: 0.9100
E

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1	Train loss: 0.4491	Validation accuracy: 0.9000
Epoch 2	Train loss: 0.1572	Validation accuracy: 0.9000
Epoch 3	Train loss: 0.1159	Validation accuracy: 0.9130
Epoch 4	Train loss: 0.1049	Validation accuracy: 0.9103
Epoch 5	Train loss: 0.0948	Validation accuracy: 0.9133
Epoch 6	Train loss: 0.1045	Validation accuracy: 0.9170
Epoch 7	Train loss: 0.0972	Validation accuracy: 0.9077
Epoch 8	Train loss: 0.0931	Validation accuracy: 0.9063
Epoch 9	Train loss: 0.0817	Validation accuracy: 0.9103
Epoch 10	Train loss: 0.0974	Validation accuracy: 0.9083
Epoch 11	Train loss: 0.0848	Validation accuracy: 0.9130
Epoch 12	Train loss: 0.0724	Validation accuracy: 0.9073
Epoch 13	Train loss: 0.0733	Validation accuracy: 0.9077
Epoch 14	Train loss: 0.0727	Validation accuracy: 0.8937
Epoch 15	Train loss: 0.0781	Validation accuracy: 0.9093
Epoch 16	Train loss: 0.0820	Validation accuracy: 0.9123
Epoch 17	Train loss: 0.0658	Validation accuracy: 0.9140
Epoch 18	Train loss: 0.0741	Validation accuracy: 0.9010
E

# VAE Train

In [16]:
def train_vae(cuda_num, check_point=5, EPOCHs=100):
    model_dir='./simulation/HAR_UCI/'
    
    device = 'cuda:%d'%cuda_num
    model = VAE_1D(9, 1536)
    model.to(device)
    
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    for epoch in tqdm(range(EPOCHs)):
        train_loss = AverageMeter()
        recon_loss = AverageMeter()
        model.train()
        for X, _ in train_loader:
            X = X.to(device)
            
            rec_enc, mean, logvar = model(X)
            
            rec_loss = ((rec_enc - X) ** 2).mean()
            prior_loss = 1 + logvar - mean.pow(2) - logvar.exp()
            prior_loss = (-0.5 * torch.sum(prior_loss)) / torch.numel(mean.data)
            err_enc = prior_loss + 5 * rec_loss
            
            train_loss.update(err_enc.item(), X.size(0))
            recon_loss.update(rec_loss.item(), X.size(0))
            
            optimizer.zero_grad()
            err_enc.backward()
            optimizer.step()
            
        print('Epoch {}\tTrain loss: {:.4f}\tRecon loss: {:.4f}'.format(epoch+1, train_loss.avg, recon_loss.avg))
        
        if epoch % check_point == 0 and epoch != 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, 'vae-epoch{}.pt'.format(epoch)))

In [17]:
train_vae(4)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1	Train loss: 1.1479	Recon loss: 0.2295
Epoch 2	Train loss: 0.5494	Recon loss: 0.1098
Epoch 3	Train loss: 0.5090	Recon loss: 0.1018
Epoch 4	Train loss: 0.5053	Recon loss: 0.1011
Epoch 5	Train loss: 0.5038	Recon loss: 0.1008
Epoch 6	Train loss: 0.5029	Recon loss: 0.1006
Epoch 7	Train loss: 0.5023	Recon loss: 0.1005
Epoch 8	Train loss: 0.5019	Recon loss: 0.1004
Epoch 9	Train loss: 0.5014	Recon loss: 0.1003
Epoch 10	Train loss: 0.5011	Recon loss: 0.1002
Epoch 11	Train loss: 0.5008	Recon loss: 0.1002
Epoch 12	Train loss: 0.5006	Recon loss: 0.1001
Epoch 13	Train loss: 0.5004	Recon loss: 0.1001
Epoch 14	Train loss: 0.5003	Recon loss: 0.1001
Epoch 15	Train loss: 0.5000	Recon loss: 0.1000
Epoch 16	Train loss: 0.5000	Recon loss: 0.1000
Epoch 17	Train loss: 0.4998	Recon loss: 0.1000
Epoch 18	Train loss: 0.4997	Recon loss: 0.0999
Epoch 19	Train loss: 0.4996	Recon loss: 0.0999
Epoch 20	Train loss: 0.4996	Recon loss: 0.0999
Epoch 21	Train loss: 0.4993	Recon loss: 0.0999
Epoch 22	Train loss: 0

# Latent Clf

In [19]:
cuda_num = 4
device = 'cuda:%d'%cuda_num

model_dir='./simulation/HAR_UCI/'
vae = VAE_1D(9, 1536)
state_dict = torch.load(os.path.join(model_dir, 'vae-epoch{}.pt'.format(95)))
vae.load_state_dict(state_dict)
vae.to(device)

VAE_1D(
  (encoder): Sequential(
    (0): Conv1d(9, 32, kernel_size=(4,), stride=(2,))
    (1): ReLU()
    (2): Conv1d(32, 64, kernel_size=(4,), stride=(2,))
    (3): ReLU()
    (4): Conv1d(64, 128, kernel_size=(4,), stride=(2,))
    (5): ReLU()
    (6): Conv1d(128, 256, kernel_size=(4,), stride=(2,))
    (7): ReLU()
    (8): Flatten()
  )
  (fc1): Linear(in_features=1536, out_features=32, bias=True)
  (fc2): Linear(in_features=1536, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1536, bias=True)
  (decoder): Sequential(
    (0): UnFlatten()
    (1): ConvTranspose1d(256, 128, kernel_size=(4,), stride=(2,))
    (2): ReLU()
    (3): ConvTranspose1d(128, 64, kernel_size=(4,), stride=(2,))
    (4): ReLU()
    (5): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), output_padding=(1,))
    (6): ReLU()
    (7): ConvTranspose1d(32, 9, kernel_size=(4,), stride=(2,))
    (8): Sigmoid()
  )
)

In [39]:
train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
vae.eval()

z_list = None
z_mean_list = None
true_label_list = None

for data, label in train_loader:
    z, z_mean, z_var = vae.encode(data.to(device))
    z_mean_list = concatenate(z_mean_list, z_mean.detach().cpu().numpy())
    z_list = concatenate(z_list, z.detach().cpu().numpy())
    true_label_list = concatenate(true_label_list, label.detach().numpy())
    
latent_loader = get_dataloader(z_mean_list, true_label_list, 256)

test_z_list = None
test_z_mean_list = None
test_true_label_list = None

for data, label in test_loader:
    z, z_mean, z_var = vae.encode(data.to(device))
    test_z_mean_list = concatenate(test_z_mean_list, z_mean.detach().cpu().numpy())
    test_z_list = concatenate(test_z_list, z.detach().cpu().numpy())
    test_true_label_list = concatenate(test_true_label_list, label.detach().numpy())
    
test_latent_loader = get_dataloader(test_z_mean_list, test_true_label_list, 256, shuffle=False)

In [40]:
class LatentClf(nn.Module):
    def __init__(self):
        super(LatentClf, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 5, padding=2, stride=2)  # in_channels=3
        self.bn1 = nn.BatchNorm1d(64, momentum=0.9)
        self.maxpool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(64, 128, 5, padding=2, stride=2)
        self.bn2 = nn.BatchNorm1d(128, momentum=0.9)
        self.fc1 = nn.Linear(128 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.LeakyReLU()
        self.bn3 = nn.BatchNorm1d(256, momentum=0.9)

    def forward(self, x):
        out = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        out = self.relu(self.bn2(self.conv2(out)))
#         print(out.shape)
        out = out.view(x.size(0),-1)
        out = self.relu(self.bn3(self.fc1(out)))
        out = self.relu(self.fc2(out))
        return out

In [42]:
l_clf = LatentClf()
l_clf.to(device)
optimizer = torch.optim.SGD(l_clf.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

best_acc = -1
best_l_clf = None

for epo in range(30):
    l_clf.train()
    lossavg = AverageMeter()
    for latent, y_hat in latent_loader:
        latent = latent.unsqueeze(1).to(device)
        
        out = l_clf(latent)
        loss = criterion(out, y_hat.long().to(device))
        lossavg.update(loss.item(), y_hat.size(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    correct = 0
    for latent, y_hat in test_latent_loader:
        out = l_clf(latent.unsqueeze(1).to(device))
        out = F.softmax(out, dim=1)
        _, pred = out.max(1)

        correct += pred.eq(y_hat.to(device)).sum().item()
        
    if best_acc < correct/test_z_list.shape[0]:
        best_acc = correct/test_z_list.shape[0]
        best_l_clf = copy.deepcopy(l_clf)
        
    print('epoch: %d\tloss: %.4f\ttestacc: %.4f' % (epo, lossavg.avg, correct/test_z_list.shape[0]))

epoch: 0	loss: 1.1204	estacc: 0.7509
epoch: 1	loss: 0.5250	estacc: 0.7849
epoch: 2	loss: 0.4259	estacc: 0.7893
epoch: 3	loss: 0.3827	estacc: 0.8045
epoch: 4	loss: 0.3427	estacc: 0.8185
epoch: 5	loss: 0.3019	estacc: 0.8066
epoch: 6	loss: 0.2746	estacc: 0.8171
epoch: 7	loss: 0.2628	estacc: 0.8005
epoch: 8	loss: 0.2279	estacc: 0.8151
epoch: 9	loss: 0.2130	estacc: 0.8086
epoch: 10	loss: 0.1931	estacc: 0.8286
epoch: 11	loss: 0.1755	estacc: 0.8134
epoch: 12	loss: 0.1711	estacc: 0.8110
epoch: 13	loss: 0.1615	estacc: 0.8246
epoch: 14	loss: 0.1489	estacc: 0.8222
epoch: 15	loss: 0.1289	estacc: 0.8286
epoch: 16	loss: 0.1241	estacc: 0.8229
epoch: 17	loss: 0.1335	estacc: 0.8276
epoch: 18	loss: 0.1222	estacc: 0.8185
epoch: 19	loss: 0.1075	estacc: 0.8205
epoch: 20	loss: 0.1015	estacc: 0.8198
epoch: 21	loss: 0.0991	estacc: 0.8229
epoch: 22	loss: 0.0942	estacc: 0.8269
epoch: 23	loss: 0.0970	estacc: 0.8276
epoch: 24	loss: 0.0952	estacc: 0.8110
epoch: 25	loss: 0.0912	estacc: 0.8079
epoch: 26	loss: 0.0868

In [43]:
torch.save(l_clf.state_dict(), os.path.join(model_dir, 'l_clf.pt'))

In [75]:
train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
device = 'cuda:%d' % CUDA_NUM

model = HARClassifier()
model.to(device)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [74]:
model_dir = './simulation/HAR_UCI/'
model_file = os.path.join(model_dir, 'comp0_0-model-epoch{}.pt'.format(99))

state_dict = torch.load(model_file)

In [73]:
'./simulation/HAR_UCI/comp0_0-model-epoch99.pt'

'./simulation/HAR_UCI/comp0_0-model-epoch99.pt'

In [72]:
import matplotlib.pyplot as plt

In [ ]:
har_train()

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1	Train loss: 1.7831	Validation accuracy: 0.4253
Epoch 2	Train loss: 1.5139	Validation accuracy: 0.4289
Epoch 3	Train loss: 0.9173	Validation accuracy: 0.6170
Epoch 4	Train loss: 0.6766	Validation accuracy: 0.6557
Epoch 5	Train loss: 0.5246	Validation accuracy: 0.6888
Epoch 6	Train loss: 0.3887	Validation accuracy: 0.7440
Epoch 7	Train loss: 0.3794	Validation accuracy: 0.7137
Epoch 8	Train loss: 0.2320	Validation accuracy: 0.8250
Epoch 9	Train loss: 0.1849	Validation accuracy: 0.8763
Epoch 10	Train loss: 0.1864	Validation accuracy: 0.8777
Epoch 11	Train loss: 0.1513	Validation accuracy: 0.8717
Epoch 12	Train loss: 0.1297	Validation accuracy: 0.8917
Epoch 13	Train loss: 0.1261	Validation accuracy: 0.8930
Epoch 14	Train loss: 0.1294	Validation accuracy: 0.8937
Epoch 15	Train loss: 0.1162	Validation accuracy: 0.8923
Epoch 16	Train loss: 0.1258	Validation accuracy: 0.8787
Epoch 17	Train loss: 0.1542	Validation accuracy: 0.8806


In [70]:
def train_iter(model, optimizer, criterion, data_loader, device, comp_ratio=0.):
    model.train()
    iteration_loss = AverageMeter()
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        
        output = model(X)
        
        loss = criterion(output, y.long())
        
        iteration_loss.update(loss.item(), X.size(0))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if comp_ratio > 0:
            idxs, lams = structured_har(model, comp_ratio)    
    if comp_ratio > 0:
        idxs, lams = structured_har(model, comp_ratio)
    return iteration_loss.avg


def test_iter(model, criterion, data_loader, device, check_adv=False, epsilon=0.3, alpha=0.0073, k=7):
    model.eval()
    normal_acc = AverageMeter()
    
    if check_adv:
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
        off_acc = AverageMeter()
    
    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)
        
        output = model(X)
        
        out = F.softmax(output, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y)
        
        acc = idx.sum().item() / X.size(0)
        normal_acc.update(acc)
        
        if check_adv:
        
            adv_x = adv.perturb(X, y.long())

            out = model(adv_x)
            out = F.softmax(out, dim=1)
            _, predicted = out.max(1)
            idx = predicted.eq(y)
            
            acc = idx.sum().item() / X.size(0)
            off_acc.update(acc)
            
    if check_adv:
        return normal_acc.avg, off_acc.avg
    else:
        return normal_acc.avg



def train(cuda_num=4, EPOCH=100, save_interval=5, comp_ratio=0.):
    model_dir = './simulation/HAR_UCI/'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    device = 'cuda:%d' % cuda_num
    
    model = HARClassifier()
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.91)
    scheduler = MultiStepLR(optimizer, milestones=[55,75,90], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in tqdm(range(EPOCH)):
        train_loss = train_iter(model, optimizer, criterion, train_loader, device)
        val_acc = test_iter(model, criterion, test_loader, device)
        scheduler.step()
        
        print('Epoch {}\tTrain loss: {:.4f}\tValidation accuracy: {:.4f}'.format(epoch+1,train_loss, val_acc))
        
        if (epoch+1) % save_interval == 0:
            torch.save(model.state_dict(),
                       os.path.join(model_dir, 'comp{}-model-epoch{}.pt'.format(str(comp_ratio*100).replace('.','_'), epoch)))
            

In [ ]:
def test_har(comp_ratio, device, model_dir='./simulation/HAR_UCI/'):
    state_dict = torch.load(os.path.join(model_dir,
                                    'comp{}-model-epoch{}.pt'.format(str(comp_ratio * 100).replace('.', '_'), epoch)))
    
    train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')
    
    model = HARClassifier()
    model.to(device)
    model.load(state_dict)
    
    acc, off_acc = test_iter(model, test_loader, device, check_adv=True)

In [14]:
acc, off_acc

(0.9113333333333333, 0.32731205673758873)

In [12]:
model_dir='./simulation/HAR_UCI/'
device = 'cuda:%d'%4
state_dict = torch.load(os.path.join(model_dir,
                                    'comp0_0-model-epoch{}.pt'.format(99)))

train_loader, test_loader = uci_har('/workspace/Dataset/TSData/uci_data/np/')

model = HARClassifier()
model.to(device)
model.load_state_dict(state_dict)

acc, off_acc = test_iter(model, test_loader, device, check_adv=True)

In [5]:
def test_iter(model, data_loader, device, check_adv=False, epsilon=0.3, alpha=0.0073, k=7):
    
    model.eval()
    normal_acc = AverageMeter()

    if check_adv:
        adv = LinfPGDAttack(model, epsilon=epsilon, alpha=alpha, k=k)
        off_acc = AverageMeter()

    for i, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        output = model(X)

        out = F.softmax(output, dim=1)
        _, predicted = out.max(1)
        idx = predicted.eq(y)

        acc = idx.sum().item() / X.size(0)
        normal_acc.update(acc)

        if check_adv:
            adv_x = adv.perturb(X, y.long())

            out = model(adv_x)
            out = F.softmax(out, dim=1)
            _, predicted = out.max(1)
            idx = predicted.eq(y)

            acc = idx.sum().item() / X.size(0)
            off_acc.update(acc)

    if check_adv:
        return normal_acc.avg, off_acc.avg
    else:
        return normal_acc.avg